# Error Analysis

Let's check out what's our model doing wrong

In [1]:
%load_ext autoreload
%autoreload 2
import os
from datetime import datetime
import fire
import torch
import pandas as pd
from torchtext import data
import torch.nn as nn
from transformers import (
    AdamW, BertForSequenceClassification, BertTokenizer,
    get_constant_schedule_with_warmup
)

from offenseval.nn import (
    Tokenizer,
    train, evaluate, train_cycle, save_model, load_model, evaluate_dataset
)
from offenseval.datasets import datasets

pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 300

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model, TEXT = load_model("../models/bert_cased.all.pt", device)
model.eval();

Load the dataset as a `pd.DataFrame` (it could be done in the previous cell but well...)

In [2]:
from offenseval.nn.evaluation import evaluate
from offenseval.nn import EvaluationReport
from tqdm.auto import tqdm

dataset_path = datasets["english"]["test"]
report = evaluate_dataset(model, TEXT, dataset_path)

report


Loading dataset...
Building iterators


Acc: 91.46% Macro F1 0.899 (P 0.860 - N 0.939)

Create dataframe with predictions, probas, and so on

In [9]:
import csv

df = pd.read_table(dataset_path, index_col=0, 
                   quoting=csv.QUOTE_NONE)

df["label"] = df["subtask_a"] == 'OFF'

df["prob"] = report.probas.numpy()
df["pred"] = df["prob"] > 0.5


In [17]:
true_pos = df[df["label"] & df["pred"]]
true_neg = df[~df["label"] & ~df["pred"]]


errors = df[df["label"] != df["pred"]]

false_pos = errors[errors["pred"]]
false_neg = errors[~errors["pred"]]

print(f"We have {len(errors)} errors ({len(false_pos)} - FP {len(false_neg)} FN)")

We have 332 errors (270 - FP 62 FN)


In [19]:
false_pos[10:100]

,tweet,subtask_a,label,prob,pred
id,,,,,
A91,in the syrupy goodness of the anime con in detroit,NOT,False,0.999511,True
A125,Call me a coward but i badly want to end this all.,NOT,False,0.998663,True
A164,They be killing people like it’s living going outta style this is crazy,NOT,False,0.996506,True
A174,@USER is God dat will punish you people I swear!!! The worst bank in the whole of Africa,NOT,False,0.999435,True
A239,@USER @USER Also fuck fat tire who the hell are you I’m drinking hurricanes,NOT,False,0.999027,True
A247,@USER yeah he manages to look ok in outfits that other ppl would look mad ugly in... his power,NOT,False,0.999062,True
A253,@USER He deserve the worst of this all,NOT,False,0.998008,True
A264,@USER MY HEADER NOW IS THE STUPID BAG AOCO GAVE ME,NOT,False,0.999545,True
A277,@USER @USER I’m gonna use that bigger gone to destroy you instead,NOT,False,0.997578,True


In [21]:
false_neg

,tweet,subtask_a,label,prob,pred
id,,,,,
A18,Crazy that as we get older and go through certain shit you just want to keep it to yourself.,OFF,True,0.029142,False
A49,what a good liar and pretender 😂🙅🏼,OFF,True,0.040723,False
A224,@USER Lol chopsticks is for lo mien and sushi ...my ass would be picking up the rice one by one 😂😂,OFF,True,0.021795,False
A328,I hate those females that think just because you stopped talking to them “it’s beef” like never that I just be to mf busy.,OFF,True,0.161206,False
A382,"men who voted yes are childish, sorry are you 17??Men, would you have a problem if a girl said if she’s not receiving head she’s not giving head?",OFF,True,0.000910,False
A430,@USER Somewhere Dr. Martin Luther King Jr. is weeping. You have turned his dream into a nightmare with your rank racism.,OFF,True,0.329463,False
A440,Oh my fucking god I just found one of the first songs I've ever written from 2012 and it's funny I can't,OFF,True,0.003278,False
A491,@USER @USER The pot calling the kettle black.... he's a disgrace to our nation.,OFF,True,0.027884,False
A507,I’m playing cup pong with Lydia and she sucks. Who wants to play me - good people only plz I need a little bit of a challenge at least smh,OFF,True,0.181421,False
